In [1]:
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import uniform

### Задание 1

In [2]:
data = pd.read_csv(datasets.load_boston()["filename"], skiprows = 1)

In [3]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
X = data.drop("MEDV", axis = 1)
Y = data["MEDV"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [5]:
models = [ 
    {"name": "Ridge", "model": Ridge(), "params": {"alpha": uniform(loc = 0, scale = 5), "solver": ["svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]}},
    {"name": "Lasso", "model": Lasso(), "params": {"alpha": uniform(loc = 0, scale = 5), "selection": ["cyclic", "random"]}},
    {"name": "ElasticNet", "model": ElasticNet(), "params": {"alpha": uniform(loc = 0, scale = 5), "selection": ["cyclic", "random"]}},
    {"name": "DecisionTreeRegressor", "model": DecisionTreeRegressor(), "params": {"splitter": ["best", "random"], "max_depth": [3, 5, 7, 9, 11], "criterion": ["mse", "friedman_mse", "mae", "poisson"]}},
    {"name": "RandomForestRegressor", "model": RandomForestRegressor(), "params": {"n_estimators": [10, 25, 50, 100, 150, 200], "max_depth": [3, 5, 7, 9, 11], "criterion": ["mse", "mae"]}},
    {"name": "GradientBoostingRegressor", "model": GradientBoostingRegressor(), "params": {"max_depth": [3, 5, 7, 9, 11], "loss": ["ls", "lad", "huber", "quantile"], "criterion": ["friedman_mse", "mse"]}},
    {"name": "KNeighborsRegressor", "model": KNeighborsRegressor(), "params": {"weights": ["uniform", "distance"], "p": [1, 2, 3], "n_neighbors": list(range(1, 30))}}
]

In [6]:
results = []

for model in models:
    results.append((model["name"], RandomizedSearchCV(model["model"], model["params"], cv = 10).fit(X_train, Y_train)))
    
for result in results:
    print(result[0], round(result[1].best_score_, 4), result[1].best_params_)

Ridge 0.7325 {'alpha': 0.5081022443679084, 'solver': 'svd'}
Lasso 0.6283 {'alpha': 1.8617473953236314, 'selection': 'cyclic'}
ElasticNet 0.7205 {'alpha': 0.04316001055922114, 'selection': 'random'}
DecisionTreeRegressor 0.7642 {'splitter': 'random', 'max_depth': 5, 'criterion': 'mse'}
RandomForestRegressor 0.8477 {'n_estimators': 50, 'max_depth': 11, 'criterion': 'mse'}
GradientBoostingRegressor 0.868 {'max_depth': 5, 'loss': 'ls', 'criterion': 'mse'}
KNeighborsRegressor 0.6489 {'weights': 'distance', 'p': 1, 'n_neighbors': 7}


Проверим лучшую модель на тестовых данных.

In [7]:
print("Точность предсказания:", round(results[5][1].best_estimator_.score(X_test, Y_test), 4))

Точность предсказания: 0.869
